In [ ]:
# Import modules
import numpy as np
import pandas as pd
import csv

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC, SVC
from sklearn.utils.class_weight import compute_class_weight
import time

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Flatten, Dropout
from keras import backend as K
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from keras.layers import Masking
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from keras.layers import LSTM, Dropout, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.losses import sparse_categorical_crossentropy
from keras.regularizers import l2

In [ ]:
# Opens file and put in dataframe.
oud_nl, label_org, nieuw_lemma, nieuw_lemma_split, geo_id  = [],[],[],[],[]
with open('brief.all.geoid.txt', 'r', encoding="utf8") as infile:
        text = infile.readlines()
        for line in text:
            words = line.split()
            if len(words) > 2:
                oud_nl.append(words[0])
                label_org.append(words[1])
                nieuw_lemma.append(" ".join(words[2:-2]))
                nieuw_lemma_split.append(words[-2])
                geo_id.append(words[-1])

df = pd.DataFrame()
df['oud_nl'], df['label_org'],df['nieuw_lemma'],df['nieuw_lemma_split'], df['geo_id'] = \
oud_nl, label_org, nieuw_lemma, nieuw_lemma_split, geo_id


In [ ]:
#splits the data in train and test
X = df[['oud_nl', 'nieuw_lemma', 'nieuw_lemma_split']]
y = df[['label_org', 'geo_id','nieuw_lemma_split']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)
X_test

,oud_nl,nieuw_lemma,nieuw_lemma_split
413157,dat,alsdat,alsdat
187250,betrouwen,betrouwen,betrouw
245043,becomt,bekomen,bekomen
333375,hooren,horen,horen
213189,hoop,hopen,hopen
...,...,...,...
371880,uijt,uitkomst,uitkomst
397914,wat,wat,wat
44894,te,te,te
50829,waar,waarvoor,waarvoor


In [ ]:
# This function predicts the translation
def predict_translation(model, tokenizer, max_len, input_sequences):
    # Pad input sequences
    padded_input_sequences = pad_sequences(input_sequences, maxlen=1, padding='post')

    # Make predictions
    predictions = model.predict(padded_input_sequences)

    # Convert predictions to class indices
    predicted_indices = np.argmax(predictions, axis=-1)

    # Convert indices to text
    inv_target_index = {v: k for k, v in tokenizer.word_index.items()}
    predicted_texts = []
    for sequence in predicted_indices:
      predicted_text = " ".join([inv_target_index[idx] for idx in sequence if idx != 0])
      predicted_texts.append(predicted_text)

    return predicted_texts

In [ ]:
# Translation model

# Take the source and target information out of dataframe.
X_train_t, X_test_t, y_train_t, y_test_t = X_train['oud_nl'], X_test['oud_nl'], y_train['nieuw_lemma_split'], y_test['nieuw_lemma_split']

# Tokenize the data
source_tokenizer_t = Tokenizer()
source_tokenizer_t.fit_on_texts(X_train_t)
X_train_seq_t = source_tokenizer_t.texts_to_sequences(X_train_t)
X_test_seq_t = source_tokenizer_t.texts_to_sequences(X_test_t)

target_tokenizer_t = Tokenizer()
target_tokenizer_t.fit_on_texts(y_train_t)
y_train_seq_t = target_tokenizer_t.texts_to_sequences(y_train_t)
y_test_seq_t = target_tokenizer_t.texts_to_sequences(y_test_t)

# Pad the data
max_len_source_t = max(len(seq) for seq in X_train_seq_t)
max_len_target_t = max(len(seq) for seq in y_train_seq_t)

X_train_pad_t = pad_sequences(X_train_seq_t, maxlen=1, padding='post')
X_test_pad_t = pad_sequences(X_test_seq_t, maxlen=1, padding='post')

y_train_pad_t = pad_sequences(y_train_seq_t, maxlen=1, padding='post')
y_test_pad_t = pad_sequences(y_test_seq_t, maxlen=1, padding='post')

model_trans = Sequential([
Embedding(input_dim=len(source_tokenizer_t.word_index) + 1, output_dim=128, input_length=1),
Bidirectional(LSTM(256, return_sequences=True)),
Dropout(0.5),# Adjusted complexity  # Adjusted dropout
TimeDistributed(Dense(len(target_tokenizer_t.word_index) + 1, activation='softmax'))])

# Compiling model
optimizer = Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
model_trans.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Saving model
model_trans.summary()
filename = 'model_trans.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model_trans.fit(X_train_pad_t, y_train_pad_t,
          epochs=15,
          batch_size=400,
          validation_data=(X_test_pad_t, y_test_pad_t),
          callbacks=[checkpoint])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 1, 128)            5399296   
                                                                 
 bidirectional_11 (Bidirect  (None, 1, 512)            788480    
 ional)                                                          
                                                                 
 dropout_11 (Dropout)        (None, 1, 512)            0         
                                                                 
 time_distributed_11 (TimeD  (None, 1, 15920)          8166960   
 istributed)                                                     
                                                                 
Total params: 14354736 (54.76 MB)
Trainable params: 14354736 (54.76 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1030/1030 [==============================] - 33s 29ms/step - loss: 3.6688 - accuracy: 0.5496 - val_loss: 2.1665 - val_accuracy: 0.7237
Epoch 2/15
1030/1030 [==============================] - ETA: 0s - loss: 1.7699 - accuracy: 0.7595
Epoch 2: val_loss improved from 2.16649 to 1.70118, saving model to model_trans.h5
1030/1030 [==============================] - 23s 23ms/step - loss: 1.7699 - accuracy: 0.7595 - val_loss: 1.7012 - val_accuracy: 0.7761
Epoch 3/15
1030/1030 [==============================] - ETA: 0s - loss: 1.3137 - accuracy: 0.8078
Epoch 3: val_loss improved from 1.70118 to 1.55970, saving model to model_trans.h5
1030/1030 [==============================] - 23s 22ms/step - loss: 1.3137 - accuracy: 0.8078 - val_loss: 1.5597 - val_accuracy: 0.7897
Epoch 4/15
1030/1030 [==============================] - ETA: 0s - loss: 1.0849 - accuracy: 0.8289
Epoch 4: val_loss improved from 1.55970 to 1.50766, saving model to model_trans.h5
1030/1030 [==============================] - 23s 22m

In [ ]:
print(y_test_t)

413157       alsdat
187250      betrouw
245043      bekomen
333375        horen
213189        hopen
            ...    
371880     uitkomst
397914          wat
44894            te
50829      waarvoor
142473    bijzonder
Name: nieuw_lemma_split, Length: 45758, dtype: object


In [ ]:
# Splits the data in subsets so program wont crash
def get_translation(X_source, target_tokenizer, source_tokenizer, max_len_target, model_trans):
  final = ""
  j = 1
  temp = []
  final_list = []
  for i in range(len(X_source)):
    temp.append(str(X_source[i]))
    if j == 10000:
      test_sequences = source_tokenizer.texts_to_sequences(temp)
      predicted_sequences = predict_translation(model_trans, target_tokenizer, max_len_target, test_sequences)
      final_list.append(predicted_sequences)
      j = 0
      temp = []
    else:
      j += 1
  test_sequences = source_tokenizer.texts_to_sequences(temp)
  predicted_sequences = predict_translation(model_trans, target_tokenizer, max_len_target, test_sequences)
  final_list.append(predicted_sequences)
  final_f_list = []
  for x in final_list:
    for y in x:
      final_f_list.append(y)

  return final_f_list

In [ ]:
X_train_t

160128        kense
21419        wenste
251899    Amsterdam
189113        schip
2511           Zijn
            ...    
259178         gaet
365838     briefien
131932          kan
146867         daer
121958        garen
Name: oud_nl, Length: 411813, dtype: object

In [ ]:
# Add translation column to  Dataframe
df['trans'] = get_translation(df['oud_nl'], target_tokenizer_t, source_tokenizer_t, max_len_target_t, model_trans)

236/236 [==============================] - 0s 2ms/step


In [ ]:
#splits the data in train and test
X = df[['oud_nl', 'nieuw_lemma', 'nieuw_lemma_split', 'trans']]
y = df[['label_org', 'geo_id','nieuw_lemma_split']]


X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X, y, test_size=0.1, random_state = 42)
X_test_l


,oud_nl,nieuw_lemma,nieuw_lemma_split,trans
413157,dat,alsdat,alsdat,dat
187250,betrouwen,betrouwen,betrouw,betrouwen
245043,becomt,bekomen,bekomen,unresolved
333375,hooren,horen,horen,horen
213189,hoop,hopen,hopen,hopen
...,...,...,...,...
371880,uijt,uitkomst,uitkomst,uit
397914,wat,wat,wat,wat
44894,te,te,te,te
50829,waar,waarvoor,waarvoor,waarin


In [ ]:
# Label Model
X_train_l, X_test_l, y_train_l, y_test_l = X_train_l['trans'], X_test_l['trans'], y_train_l['label_org'], y_test_l['label_org']

# Tokenize data
source_tokenizer_l = Tokenizer()
source_tokenizer_l.fit_on_texts(X_train_l)
X_train_seq_l = source_tokenizer_l.texts_to_sequences(X_train_l)
X_test_seq_l = source_tokenizer_l.texts_to_sequences(X_test_l)

target_tokenizer_l = Tokenizer()
target_tokenizer_l.fit_on_texts(y_train_l)
y_train_seq_l = target_tokenizer_l.texts_to_sequences(y_train_l)
y_test_seq_l = target_tokenizer_l.texts_to_sequences(y_test_l)


# Pad the data
max_len_source = max(len(seq) for seq in X_train_seq_l)
max_len_target = max(len(seq) for seq in y_train_seq_l)

X_train_pad_l = pad_sequences(X_train_seq_l, maxlen=1, padding='post')
X_test_pad_l = pad_sequences(X_test_seq_l, maxlen=1, padding='post')

y_train_pad_l = pad_sequences(y_train_seq_l, maxlen=1, padding='post')
y_test_pad_l = pad_sequences(y_test_seq_l, maxlen=1, padding='post')

#y_train_reshaped = y_train_pad_l.reshape((y_train_pad.shape[0], y_train_pad.shape[1]))
#y_test_reshaped = y_test_pad.reshape((y_test_pad.shape[0], y_test_pad.shape[1]))

# Model
model_label = Sequential([
Embedding(input_dim=len(source_tokenizer_l.word_index) + 1, output_dim=128, input_length=1),
Bidirectional(LSTM(256, return_sequences=True)),
Dropout(0.5),# Adjusted complexity  # Adjusted dropout
TimeDistributed(Dense(len(target_tokenizer_l.word_index) + 1, activation='softmax'))])

# Mask layer for more focus on neloc label
index_neloc = target_tokenizer_l.word_index['neloc']
sample_weights = np.ones(len(y_train_pad_l))
mask = (y_train_pad_l[:, 0] == index_neloc)
sample_weights[mask] = 2.0

# Compile model
optimizer = Adam(learning_rate=0.001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
model_label.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_label.summary()

filename = 'model_label.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model_label.fit(X_train_pad_l, y_train_pad_l,
          epochs=10,
          batch_size=400,
          validation_data=(X_test_pad_l, y_test_pad_l),
          sample_weight=sample_weights,
          callbacks=[checkpoint])



Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 1, 128)            1761920   
                                                                 
 bidirectional_12 (Bidirect  (None, 1, 512)            788480    
 ional)                                                          
                                                                 
 dropout_12 (Dropout)        (None, 1, 512)            0         
                                                                 
 time_distributed_12 (TimeD  (None, 1, 56)             28728     
 istributed)                                                     
                                                                 
Total params: 2579128 (9.84 MB)
Trainable params: 2579128 (9.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
1030

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1030/1030 [==============================] - ETA: 0s - loss: 0.4230 - accuracy: 0.8784
Epoch 2: val_loss did not improve from 0.55712
1030/1030 [==============================] - 6s 6ms/step - loss: 0.4230 - accuracy: 0.8784 - val_loss: 0.5787 - val_accuracy: 0.8240
Epoch 3/10
1030/1030 [==============================] - ETA: 0s - loss: 0.4006 - accuracy: 0.8813
Epoch 3: val_loss did not improve from 0.55712
1030/1030 [==============================] - 6s 6ms/step - loss: 0.4006 - accuracy: 0.8813 - val_loss: 0.5790 - val_accuracy: 0.8232
Epoch 4/10
1030/1030 [==============================] - ETA: 0s - loss: 0.3941 - accuracy: 0.8818
Epoch 4: val_loss did not improve from 0.55712
1030/1030 [==============================] - 6s 6ms/step - loss: 0.3941 - accuracy: 0.8818 - val_loss: 0.5690 - val_accuracy: 0.8230
Epoch 5/10
1030/1030 [==============================] - ETA: 0s - loss: 0.3910 - accuracy: 0.8822
Epoch 5: val_loss improved from 0.55712 to 0.55250, saving model to model_label

In [ ]:
print(y_test_l)
print(X_test_l)

413157    CON
187250    NOU
245043    VRB
333375    VRB
213189    VRB
         ... 
371880    NOU
397914    PRN
44894     ADP
50829     ADV
142473    ADJ
Name: label_org, Length: 45758, dtype: object
413157           dat
187250       betrouw
245043    unresolved
333375         horen
213189         hopen
             ...    
371880           uit
397914           wat
44894             te
50829           waar
142473     bijzonder
Name: trans, Length: 45758, dtype: object


In [ ]:
# Calculates Precision, Recall and F1-score
total, tp, fp, fn = 0, 0, 0, 0
test_sequences = source_tokenizer_l.texts_to_sequences(X_test_l)
y_pred = predict_translation(model_label, target_tokenizer_l, max_len_target, test_sequences)

for x, y in zip(y_pred, y_test_l):
    if y.lower() == 'neloc' and x.lower() == 'neloc':
        total += 1
        tp += 1
    elif x.lower() == 'neloc':
        if y.lower() != 'neloc':
            fn += 1
            total += 1
    elif y.lower() == 'neloc':
        if x.lower() != 'neloc':
            total += 1
            fp += 1
pre = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * ((pre * rec) / (pre + rec))
print('precision: ' + str((tp / (tp + fp))))
print('recall: ' + str((tp / (tp + fn))))
print('accuracy: ' + str((tp / total)))
print('f1 score: ' + str(f1))
print(fp)
print(total)
print(tp / total)


1430/1430 [==============================] - 4s 2ms/step
precision: 0.6389324960753532
recall: 0.8532494758909853
accuracy: 0.5756718528995757
f1 score: 0.7307001795332136
230
707
0.5756718528995757


In [ ]:
# Add label column to dataset
df['label_pred'] = get_translation(df['trans'], target_tokenizer_l, source_tokenizer_l, max_len_target, model_label)

236/236 [==============================] - 1s 2ms/step


In [ ]:
df

,oud_nl,label_org,nieuw_lemma,nieuw_lemma_split,geo_id,trans,label_pred
0,te,ADP,te,te,1,te,adp
1,bestellen,VRB,bestellen,bestellen,1,bestellen,vrb
2,aen,ADP,aan,aan,1,aan,adp
3,mester,NOU,meester,meester,1,meester,nou
4,riemer,NEPER,Riemer,Riemer,1,riemer,neper
...,...,...,...,...,...,...,...
457566,St,NELOC,Sint Maarten,Sint,7609695,sint,neloc
457567,Martin,NELOC,Sint Maarten,Maarten,7609695,maarten,neloc
457568,11de,NUM,elfde,elf,1,elf,num
457569,November,NOU,november,november,1,november,nou


In [ ]:
#splits the data in train and test
X = df[['oud_nl', 'nieuw_lemma', 'nieuw_lemma_split', 'trans', 'label_pred']]
y = df[['label_org', 'geo_id','nieuw_lemma_split']]

X = df[['trans', 'oud_nl', 'label_pred']] # Features
y = df['label_org']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42, shuffle=False)


In [ ]:

X_test_d

,trans,oud_nl
245630,een,een
364064,te,te
431672,als,als
129779,komen,koomen
425771,verhopen,verhoop
...,...,...
440919,visser,visser
290002,wij,ons
75409,hebben,heb
311867,zijn,waer


In [ ]:
# Detection model

# Model setup so it can take two datasets for source
print("Step 2(1/2) Splitting data")
print("Step 2(2/2) Splitting data")
model2 = Pipeline([
    ('tfidf', ColumnTransformer(
        transformers=[
            ('trans', TfidfVectorizer(), 'trans'),
            ('label_pred', TfidfVectorizer(), 'label_pred'),
            ('oud_nl', TfidfVectorizer(), 'oud_nl')
        ],
        remainder='passthrough'
    )),
    ('svc', LinearSVC())
])



# Train the model
print("Step 3(1/2) Training model")
start = time.time()

model2.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {stop - start}s")
print("Step 3(2/2) Training model")

# Predictions on the test set
y_pred = model2.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
total, tp, fp, fn = 0, 0, 0, 0

# This calculates precision, recall, f1-score.
for x, y in zip(y_pred, y_test):
    if y.lower() == 'neloc' and x.lower() == 'neloc':
        total += 1
        tp += 1
    elif x.lower() == 'neloc':
        if y.lower() != 'neloc':
            fn += 1
            total += 1
    elif y.lower() == 'neloc':
        if x.lower() != 'neloc':
            total += 1
            fp += 1
pre = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * ((pre * rec) / (pre + rec))
print('precision: ' + str((tp / (tp + fp))))
print('recall: ' + str((tp / (tp + fn))))
print('accuracy: ' + str((tp / total)))
print('f1 score: ' + str(f1))
print(fp)
print(total)
print(tp / total)


Step 2(1/2) Splitting data
Step 2(2/2) Splitting data
Step 3(1/2) Training model
Training time: 701.8042795658112s
Step 3(2/2) Training model
Accuracy: 0.88
precision: 0.8496
recall: 0.885
accuracy: 0.7651296829971181
f1 score: 0.8669387755102042
94
694
0.7651296829971181


In [ ]:
import geocoder
import re
translation, labels, gold_std, i = [], [], [], 0
temp = ""

for trans, label, geoID in zip(df['trans'], df['label_org'], df['geo_id']):
  if i == 200:
    break
  pattern = r'(?<=[a-z])(?=[A-Z])'

  trans = re.sub(pattern, ' ', trans)

  if trans.lower() in ['demerary']:
          trans = 'demerara'
  if label == 'NELOC':
    i+=1
    if trans.lower() in ['st', 'sint', 'saint', 'domingo', 'el', 'cap']:
      if trans.lower() == 'el':
        temp = trans
        temp1 = label
        temp2 = geoID
      else:
        temp = trans + ' '
        temp1 = label
        temp2 = geoID

    else:
      if len(temp) > 1:
        if trans.lower() in ['kitts']:
          trans += ' and nevis'
        if trans.lower() in ['vincent']:
          trans += ' en de Grenadines'
        translation.append(temp + trans)
        labels.append(label)
        gold_std.append(geoID)
        temp = ""
      else:
        translation.append(trans)
        labels.append(label)
        gold_std.append(geoID)

  if len(temp) > 1 and label != 'NELOC':
      translation.append(temp)
      labels.append(temp1)
      gold_std.append(temp2)
      temp, temp1, temp2 = "", "", ""


print(len(labels))
print(len(translation))
print(len(gold_std))
print(translation)

data1, data2, data3, i = [],[],[],0

for trans, label, geoID in zip(translation, labels, gold_std):
  check = False
  if label == 'NELOC':
    g = geocoder.geonames(trans, key='pieterjansma', maxRows=5)

    if g.geonames_id == None:
      data1.append(trans)
      data2.append(0)
      data3.append(geoID)

    for x in g:
      if x.description == 'capital of a political entity':
        data1.append(trans)
        data2.append(g.geonames_id)
        data3.append(geoID)
        check = True
        break

    if check == False and g.geonames_id != None:
       for x in g:
          if x.description == 'island':
            data1.append(trans)
            data2.append(g.geonames_id)
            data3.append(geoID)
            check = True
            break

       if check == False:
          data1.append(trans)
          data2.append(g.geonames_id)
          data3.append(geoID)
          check = False


174
174
174
['amsterdam', 'velerlei', 'frankrijk', 'amsterdam', 'unresolved', 'klundert', 'izmir', 'rotterdam', 'rotterdam', 'de', 'haag', 'rotterdam', 'rotterdam', 'van', 'rijn', 'sint maarten', 'amsterdam', 'amsterdam', 'sint maarten', 'sint eustatius', 'saint nevis', 'antigua', 'barbados', 'tobago', 'granada', 'curaçao', 'bequia', 'sint vincent en de Grenadines', 'dominique', 'guadeloupe', 'grande', 'unresolved', 'unresolved', 'sint santo', 'jamaica', 'sint eustatius', 'sint maarten', 'amsterdam', 'klein', 'wittenburgerstraat', 'keizersdwarsstraat', 'amsterdam', 'sint eustatius', 'unresolved', 'westindië', 'granada', 'koffie', 'linde', 'prinsengracht', 'amsterdam', 'sint eustatius', 'amsterdam', 'scholzendijk', 'rotterdam', 'sint eustatius', 'rotterdam', 'rotterdam', 'rotterdam', 'voorburg', 'wal', 'hoog', 'leidsestraat', 'amsterdam', 'amsterdam', 'demerara', 'demerara', 'amsterdam', 'demerara', 'vlissingen', 'zeeland', 's', 'eustatius', 's', 'eustatius', 'essequibo', 'berbice', 'si

In [ ]:
tp, total = 0,0
print(data2)
print(len(data1))
print(len(data2))
print(len(data3))
for x, y, z in zip(data1, data2, data3):
    if str(z) == str(y):
        tp += 1
        total += 1
    else:
      total +=1
      print(x)
      print(y)
      print(z)


for x,y in zip(gold_std, data3):
  if x != y:
    print(x)
    print(y)

print(tp/total)

[2759794, 0, 3017382, 2759794, 0, 2752600, 311046, 2747891, 2747891, 3536729, 2747373, 2747891, 2747891, 298117, 2759875, 7609695, 2759794, 2759794, 7609695, 7626844, 3575174, 3599699, 3374084, 3573606, 3580239, 7626836, 3577956, 3577815, 3575830, 3579143, 3462114, 0, 0, 3513426, 3489940, 7626844, 7609695, 2759794, 2774021, 0, 0, 2759794, 7626844, 0, 940335, 3580239, 3383831, 2016061, 6484621, 2759794, 7626844, 2759794, 0, 2747891, 7626844, 2747891, 2747891, 2747891, 2745321, 2800481, 2753674, 0, 2759794, 2759794, 3378644, 3378644, 2759794, 3378644, 2745392, 2744011, 3579925, 7626844, 3579925, 7626844, 3375463, 3378840, 7626844, 2759881, 0, 0, 2759794, 2759881, 0, 0, 2759794, 7626844, 2759794, 2786634, 2751773, 7626844, 7626844, 3570311, 0, 7626844, 7626844, 965228, 1269750, 0, 0, 2759794, 7626844, 2755003, 1260086, 2643743, 2759794, 7626844, 2759794, 3580239, 0, 3374084, 7626844, 2016061, 0, 0, 0, 2759794, 2924477, 2759794, 0, 0, 1522237, 3536729, 1818192, 2759794, 2924477, 2623032, 6

In [ ]:
df[6100:6150]

,oud_nl,label_org,nieuw_lemma,nieuw_lemma_split,geo_id,trans,label_pred
6100,PBlandauw,NEPER_NEPER,Pieter_Blandauw,Pieter_Blandauw,1,blandauw,neper
6101,Wel,ADJ,weledelgeboren,weledelgeboren,1,wel,adv
6102,Edele,ADJ,weledelgeboren,weledelgeboren,1,edel,nou
6103,Geb,ADJ,weledelgeboren,weledelgeboren,1,weledelgeboren,adj
6104,Heer,NOU,heer,heer,1,heer,nou
6105,Den,ART,de,de,1,de,art
6106,Heer,NOU,heer,heer,1,heer,nou
6107,J,NEPER,J,J,1,j,neper
6108,G,NEPER,G,G,1,g,neper
6109,Decker,NEPER,Decker,Decker,1,decker,neper
